In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/raw_data

/content/drive/My Drive/Colab Notebooks/raw_data


In [ ]:
train = pd.read_csv("fulltrain.csv",header=None)
text = train.iloc[:,1]
test = pd.read_csv("balancedtest.csv",header=None)
t_text = test.iloc[:,1]

### Bert init steps


In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q tf-models-official==2.7.0

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

## Preprocessing for Text

In [ ]:
text=train['Text']
t_text=test['Text']

In [ ]:
#Delete HTML
from bs4 import BeautifulSoup
text = text.apply(lambda x: BeautifulSoup(x,'html.parser').get_text())
t_text = t_text.apply(lambda x: BeautifulSoup(x,'html.parser').get_text())

In [ ]:
#Remove emoji
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
text=text.apply(lambda x: remove_emoji(x))
t_text=t_text.apply(lambda x: remove_emoji(x))

In [ ]:
#Transform to lower letter
text = text.apply(lambda x: x.lower())
t_text = t_text.apply(lambda x: x.lower())

In [ ]:
# segment articles to shorter sentences
import nltk
nltk.download('punkt')
text = text.apply(lambda x: nltk.tokenize.sent_tokenize(x))
t_text = t_text.apply(lambda x: nltk.tokenize.sent_tokenize(x))
text = text.apply(lambda x: [" ".join(x[i:i+3]) for i in range(0,len(x),3)])
t_text = t_text.apply(lambda x: [" ".join(x[i:i+3]) for i in range(0,len(x),3)])
train['Text'] = text
test['Text'] = t_text
train = train.explode('Text')
test = test.explode('Text')

In [ ]:
text=train['Text']
t_text=test['Text']
#Remove punctuation
import re
text = text.apply(lambda x: re.sub(r'[^\w\s]', '', x))
t_text = t_text.apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [ ]:
#Substitute number
import inflect
def to_digit(digit):
    i = inflect.engine()
    if digit.isdigit():
        output = i.number_to_words(digit)
    else:
        output = digit
    return output
text = text.apply(lambda x: to_digit(x))
t_text = t_text.apply(lambda x: to_digit(x))

In [ ]:
#Remove the stopwords
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
text=text.apply(lambda sen:" ".join(x for x in sen.split() if x not in stop))
t_text=t_text.apply(lambda sen:" ".join(x for x in sen.split() if x not in stop))

In [ ]:
#Remove the frequency words
freq=pd.Series(' '.join(text).split()).value_counts()[:10]
freq=list(freq.index)
text=text.apply(lambda sen:' '.join(x for x in sen.split() if x not in freq))
t_text=t_text.apply(lambda sen:' '.join(x for x in sen.split() if x not in freq))

In [ ]:
# Remove the scarce word
freq = pd.Series(' '.join(text).split()).value_counts()[-10:]
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
t_text = t_text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
t_text.head()

0    many actors seem content churn performances qu...
0    ive ive done bridges rattling laundry list fil...
0    waste anything else powerful though made clear...
0    towels bathrobes carefully placed plants added...
0    part jeffs fans nothing supportive wow whether...
Name: Text, dtype: object

In [ ]:
#Noise Removal
def text_cleaner(text):
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
        {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''}  # remove spaces at the beginning
    ]
    for rule in rules:
      for (k, v) in rule.items():
        regex = re.compile(k)
        text = regex.sub(v, text)
      text = text.rstrip()
    return text.lower()

text=text.apply(lambda x: text_cleaner(x))
t_text=t_text.apply(lambda x: text_cleaner(x))

In [ ]:
#Stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
text=text.apply(lambda x:" ".join(ps.stem(word) for word in x.split()))
t_text=t_text.apply(lambda x:" ".join(ps.stem(word) for word in x.split()))

In [ ]:
#Lemmatization
from textblob import Word
import nltk
nltk.download('wordnet')
text=text.apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
t_text=t_text.apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
## better for next time run
train['Text'] = text
test['Text'] = t_text
train.to_csv('train_pre.csv', index=False)
test.to_csv('test_pre.csv', index=False)

## 3. Model fitting

### previous self-trained bert encoder

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.models import Model 
import bert
max_seq_length = 512 
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=True)
# pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
# model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

In [ ]:
from bert.tokenization import FullTokenizer
tokenizer = FullTokenizer(vocab_file, do_lower_case)
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
tokens,masks,segments = bert_encode(text, tokenizer)

In [ ]:
t_tokens,_,_ = bert_encode(t_text, tokenizer)

### Final pre-trained bert encoder

In [ ]:
train_pre = pd.read_csv("train_pre.csv",header=0)
train_pre.dropna(inplace=True)
train_pre.reset_index(inplace=True,drop=True)
# load sections of text to avoid ram crash
text = train_pre.iloc[415000:,1]

test_pre = pd.read_csv("test_pre.csv",header=0)
test_pre.dropna(inplace=True)
test_pre.reset_index(inplace=True,drop=True)
t_text = test_pre['Text']

#print("Train file start encode")
#text_prepro = text.apply(lambda x: bert_preprocess_model([x]))
#text_encode = text_prepro.apply(lambda x: bert_model(x)["pooled_output"].numpy())
## initial saving
#text_encode.to_csv('train_encode.csv',index=False)
## continue saving
#text_encode.to_csv('train_encode.csv', mode='a',index=False, header=False)
#print("Complete")

#print("Test file start encode")
#t_text_prepro = t_text.apply(lambda x: bert_preprocess_model([x]))
#t_text_prepro =  bert_preprocess_model([t_text])
#t_text_encode = t_text_prepro.apply(lambda x: bert_model(x)["pooled_output"].numpy())
#t_text_encode.to_csv('test_encode.csv', index=False)
#print("Completed")

In [ ]:
train_pre = pd.read_csv("train_pre.csv",header=0)
train_pre.dropna(inplace=True)
y_train = train_pre['Verdict']
train_enc = pd.read_csv("train_encode.csv", header=0)['Text']
tmp = []
for i in train_enc:
    data = np.array(i[2:-2].split(), dtype='float')
    tmp.append(data)
train_enc = pd.DataFrame(tmp)
test_pre = pd.read_csv("test_pre.csv",header=0)
test_pre.dropna(inplace=True)
y_test = test_pre['Verdict']
test_enc = pd.read_csv("test_encode.csv", header=0)['Text']
tmp = []
for i in test_enc:
    data = np.array(i[2:-2].split(), dtype='float')
    tmp.append(data)
test_enc = pd.DataFrame(tmp)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_recall_fscore_support

clf5 = LogisticRegression(C=40, solver='lbfgs',max_iter=5000)
clf5.fit(train_enc, y_train)
predictions5 = clf5.predict(test_enc)
score5 = precision_recall_fscore_support(y_test, predictions5, average='macro')
print (score5, clf5.score(test_enc, y_test))